In [1]:
from ds_utils.imports import *

Using Theano backend.


In [2]:
# seasons = np.arange(1, 20)
seasons = np.arange(1, 10)

data_seasons = []
for season in seasons:
    data_season = pd.read_csv('https://github.com/BobAdamsEE/SouthParkData/raw/master/Season-%d.csv' % season)
    data_seasons.append(data_season)

data = pd.concat(data_seasons)

In [3]:
text = ' '.join([line[:-1] for line in data.Line])
print(text[:1000])

School day, school day, teacher's golden ru... Ah, damn it! My little brother's trying to follow me to school again. Zeeponanner. Ike, you can't come to school with me.  Yeah, go home you little dildo. Dude, don't call my brother a dildo! What's a dildo? Well, I don't know...  and I'll bet Cartman doesn't know either! I know what it means! Well, what? I'm not telling you. What's a dildo, Kenny? (It's a giant stick that goes inside the mom's vagina)  He-yeah, that's what Kyle's little brother is all right!  Ow!  Dude, that kicks ass! Yeah, check this one out. Ready, Ike? Kick the baby! Don't kick the baby. Kick the baby.  Whoa, Cartman! Looks like you didn't get much sleep last night. That's 'cause I was having these... bogus nightmares. Really? What about? Well, I dreamt that I was lying in my bed...  in the dark, when all of a sudden this bright blue light filled the room.  Then slowly my bedroom door begin to open,  and the next thing I remember, I was being drug through a hallway.  

In [4]:
chars = list(set(text))
vocab_size = len(chars)+1

char_ind = dict((i, c) for i, c in enumerate(chars))
ind_char = dict((c, i) for i, c in enumerate(chars))

idx = [ind_char[c] for c in text]

In [5]:
cs = 8

In [6]:
c_in_dat = [[idx[i+n] for i in range(0, len(idx)-1-cs, cs)]
            for n in range(cs)]

In [7]:
c_out_dat = [idx[i+cs] for i in range(0, len(idx)-1-cs, cs)]

In [8]:
xs = [np.stack(c[:-2]) for c in c_in_dat]

In [9]:
y = np.stack(c_out_dat[:-2])

In [10]:
n_fac = 42

In [11]:
def embedding_input(name, n_in, n_out):
    inp = keras.layers.Input(shape=(1,), dtype='int64', name=name+'_in')
    emb = keras.layers.Embedding(n_in, n_out, input_length=1, name=name+'_emb')(inp)
    return inp, keras.layers.Flatten()(emb)

In [12]:
c_ins = [embedding_input('c'+str(n), vocab_size, n_fac) for n in range(cs)]

In [13]:
n_hidden = 256

In [14]:
dense_in = keras.layers.Dense(n_hidden, activation='relu')
dense_hidden = keras.layers.Dense(n_hidden, activation='relu', init='identity')
dense_out = keras.layers.Dense(vocab_size, activation='softmax')

/opt/conda/lib/python3.6/site-packages/Keras-2.0.0-py3.6.egg/keras/legacy/interfaces.py:86: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(256, activation="relu", kernel_initializer="identity")`
  '` call to the Keras 2 API: ' + signature)


In [15]:
hidden = dense_in(c_ins[0][1])

In [16]:
for i in range(1,cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = keras.layers.merge([c_dense, hidden])

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:4: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `sum`, `concatenate`, etc.
/opt/conda/lib/python3.6/site-packages/Keras-2.0.0-py3.6.egg/keras/legacy/layers.py:456: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [17]:
c_out = dense_out(hidden)

In [29]:
model = keras.models.Model([c[0] for c in c_ins], c_out)
model.compile(
    loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam())

In [19]:
model.fit(xs, y, batch_size=64, epochs=12)

Epoch 1/12
 17792/314184 [>.............................] - ETA: 65s - loss: 2.9780 

limit_output extension: Maximum message size of 10000 exceeded with 10092 characters

In [20]:
model.save('models/southpark_rnn.h5')

In [27]:
def get_next(inp):
    idxs = [np.array(ind_char[c])[np.newaxis] for c in inp]
    p = model.predict(idxs)
    return chars[np.argmax(p)]

In [63]:
for i in range(100):
    n = np.random.choice(len(text))
    str_piece = text[n:n + 8]
    str_piece_context = text[n - 10:n + 8 + 5]
    print(str_piece_context)
    print(10 * ' ' + str_piece + str(get_next(str_piece)))
    print(10 * '-')

e crunch. They're gaini
          They're a
----------
y minute. Come on, you'
          Come on, 
----------
sing, it's because they
           because 
----------
's been a long night. W
          long nige
----------
ere here? Sure. That cr
          Sure. The
----------
 are putting my life an
          ng my lin
----------
week. At first I though
          irst I th
----------
ly one thing left to do
          ng left m
----------
Nnno, Skyler, I'm embar
          er, I'm n
----------
 Her boyfriend's here! 
          iend's ha
----------
me a best-selling autho
          selling a
----------
re so hot  on that show
           on that 
----------
telling me what your he
           what you
----------
guys smell that? Smell 
           that? St
----------
lestered me."  And then
          e."  And 
----------
fice himself in case so
          lf in car
----------
by Jones meant when he 
          eant whe 
----------
 The lava's comin' righ
          s comin' 
----------
? We are N